<h1 align="center">Exists2025:  Sexism Identification in Twitter</h1>
<h2 align="center">Task 1. Fine-tuning for binary classification
<h3 style="display:block; margin-top:5px;" align="center">ETSInf. Universitat Politècnica de València</h3>
<br>

## Libraries

In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import  AutoTokenizer, AutoModelForSequenceClassification,  Trainer, TrainingArguments,  EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import random
import os
import pandas as pd
import json
import sys
import tempfile
import time
import re
import emoji

#Importing the required modules to use the ICM measure
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.metrics.metricfactory import MetricFactory
from pyevall.reports.reports import PyEvALLReport
from pyevall.utils.utils import PyEvALLUtils
from functools import partial

from readerEXIST2025 import EXISTReader
from create_submision_folder import create_submision_file

## Read data

In [2]:
url = r"\bhttps?://[\w.-]+(?:\.[a-zA-Z]{2,})?(?:/\S*/?)?"

emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
pattern_emoji = "(" + "|".join(re.escape(u) for u in emojis) + ")"

pattern_all = r"|".join(
    [
        url,
        pattern_emoji,
    ]
)

re_all = re.compile(pattern_all, re.U)

In [3]:
file_train = "data/EXIST 2025 Tweets Dataset/training/EXIST2025_training.json"
file_dev = "data/EXIST 2025 Tweets Dataset/dev/EXIST2025_dev.json"
file_test = "data/EXIST 2025 Tweets Dataset/test/EXIST2025_test_clean.json"

reader_train = EXISTReader(file_train)
reader_dev = EXISTReader(file_dev)
reader_test = EXISTReader(file_test, is_test=True)

EnTrainTask3, EnDevTask3, EnTestTask3 = reader_train.get(lang="EN", subtask="3", regular_exp=re_all, preprocess=True), reader_dev.get(lang="EN", subtask="3", regular_exp=re_all, preprocess=True), reader_test.get(lang="EN", subtask="3", regular_exp=re_all, preprocess=True)
SpTrainTask3, SpDevTask3, SpTestTask3 = reader_train.get(lang="ES", subtask="3", regular_exp=re_all, preprocess=True), reader_dev.get(lang="ES", subtask="3", regular_exp=re_all, preprocess=True), reader_test.get(lang="ES", subtask="3", regular_exp=re_all, preprocess=True)

In [4]:
EnTrainTask3[2].head()

1    [NO, MISOGYNY-NON-SEXUAL-VIOLENCE, OBJECTIFICA...
2    [NO, MISOGYNY-NON-SEXUAL-VIOLENCE, OBJECTIFICA...
5    [IDEOLOGICAL-INEQUALITY, MISOGYNY-NON-SEXUAL-V...
6            [OBJECTIFICATION, STEREOTYPING-DOMINANCE]
7                         [NO, IDEOLOGICAL-INEQUALITY]
Name: label3, dtype: object

Ahora son 6 labels, incluyendo al NO.

## ICM Wrapper

In [5]:
def ICMWrapper(pred, labels, multi=False,ids=None):
    test = PyEvALLEvaluation()
    metrics=[MetricFactory.ICM.value]
    params= dict()
    fillLabel=None
    if multi:
        params[PyEvALLUtils.PARAM_REPORT]="embedded"
        hierarchy={"True":['IDEOLOGICAL-INEQUALITY', 'STEREOTYPING-DOMINANCE', 'MISOGYNY-NON-SEXUAL-VIOLENCE', 'OBJECTIFICATION', 'SEXUAL-VIOLENCE'],
        "False":[]}
        params[PyEvALLUtils.PARAM_HIERARCHY]=hierarchy
        fillLabel = lambda x: ["False"] if len(x)== 0 else x
    else:
        params[PyEvALLUtils.PARAM_REPORT]="simple"
        fillLabel = lambda x: str(x)


    truth_name, predict_name=None, None
    if ids is None:
        ids=list(range(len(labels)))

    with tempfile.NamedTemporaryFile(mode='w', delete=False, encoding='utf-8') as truth:
        truth_name=truth.name
        truth_df=pd.DataFrame({'test_case': ['EXIST2025']*len(labels),
                        'id': [str(x) for x in ids],
                        'value': [fillLabel(x) for x in labels]})
        if multi==True:
            truth_df=truth_df.astype('object')
        truth.write(truth_df.to_json(orient="records"))

    with  tempfile.NamedTemporaryFile(mode='w', delete=False) as predict:
        predict_name=predict.name
        predict_df=pd.DataFrame({'test_case': ['EXIST2025']*len(pred),
                        'id': [str(x) for x in ids],
                        'value': [fillLabel(x) for x in pred]})
        if multi==True:
            predict_df=predict_df.astype('object')
        predict.write(predict_df.to_json(orient="records"))

    report = test.evaluate(predict_name, truth_name, metrics, **params)
    os.unlink(truth_name)
    os.unlink(predict_name)

    icm = None
    if 'metrics' in report.report:
        if 'ICM' in report.report["metrics"]: icm=float(report.report["metrics"]['ICM']["results"]["average_per_test_case"])
    return icm

## Set seed

In [6]:
def set_seed(seed=1234):
    """
    Sets the seed to make everything deterministic, for reproducibility of experiments
    Parameters:
    seed: the number to set the seed to
    Return: None
    """
    # Random seed
    random.seed(seed)
    # Numpy seed
    np.random.seed(seed)
    # Torch seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    # os seed
    os.environ['PYTHONHASHSEED'] = str(seed)

## Dataset class

In [7]:
class SexismDatasetMulti(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_len=128, pad="max_length", trunc=True,rt='pt'):
        self.texts = texts.tolist()
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.trunc = trunc
        self.rt = rt


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,padding=self.pad, truncation=self.trunc,
            return_tensors=self.rt
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float),
            'id': torch.tensor(self.ids[idx], dtype=torch.long)}

## Metrics

In [8]:
def compute_metrics_3(pred, lencoder):
    labels = pred.label_ids
    #preds = pred.predictions.argmax(-1)
    preds = torch.sigmoid(torch.tensor(pred.predictions)).numpy()
    preds_binary = (preds >= 0.5).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds_binary, average=None, zero_division=0
    )
    acc = accuracy_score(labels, preds_binary)
    icm= ICMWrapper(lencoder.inverse_transform(preds_binary), lencoder.inverse_transform(labels), multi=True)
    # Macro averages
    precision_macro = np.mean(precision)
    recall_macro = np.mean(recall)
    f1_macro = np.mean(f1)
    metrics = {}
    metrics.update({
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'ICM': icm
    })
    return metrics

## Pipeline

In [9]:
def sexism_classification_pipeline_task3(trainInfo, devInfo, testInfo=None, model_name='roberta-base', nlabels=5, ptype="multi_label_classification", **args):
    # Model and Tokenizer
    labelEnc= MultiLabelBinarizer()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype, 
        ignore_mismatched_sizes=args.get("ignore_mismatched_sizes", False)
        )

    # Prepare datasets
    train_dataset = SexismDatasetMulti(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDatasetMulti(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        #warmup_steps=args.get('warmup_steps', 500),
        warmup_ratio=args.get("warmup_ratio", 0.1),
        weight_decay=args.get('weight_decay',0.01),
        lr_scheduler_type=args.get("scheduler_type", "linear"),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        save_total_limit=args.get('save_total_limit', 1),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        metric_for_best_model=args.get('metric_for_best_model',"ICM")
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        #compute_metrics=compute_metrics_3,
        compute_metrics = partial(compute_metrics_3, lencoder=labelEnc),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    if testInfo is not None:
        # Prepare test dataset for prediction
        test_dataset = SexismDatasetMulti(testInfo[1], [[0]*nlabels] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)
        output_type = args.get("output_type", "hard")
        # Predict test set labels
        predictions = trainer.predict(test_dataset)
        #predicted_labels = np.argmax(predictions.predictions, axis=1)
        probabilities = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()

        if output_type == "hard":
            predicted_labels = (probabilities >= 0.5).astype(int)
            # Create submission DataFrame
            labels = labelEnc.inverse_transform(predicted_labels)
            list_labels = []
            for t in labels:
                aux = []
                for e in t:
                    aux.append(e)
                list_labels.append(aux)

            submission_df = pd.DataFrame({
                'id': testInfo[0],
                'label': list_labels,
                "test_case": ["EXIST2025"]*len(predicted_labels)
            })

        elif output_type == "soft":
            submission_data = {
                'id': testInfo[0],
                "test_case": ["EXIST2025"] * len(probabilities)
            }
            list_probabilities = []
            for i in range(probabilities.shape[0]):
                value = {}
                for j in range(probabilities.shape[1]):
                    c = labelEnc.classes_[j]
                    value[c] = probabilities[i,j].item()
                list_probabilities.append(value)

            submission_data["value"] = list_probabilities
            submission_df = pd.DataFrame(submission_data)

        language = args.get("language", "english")
        submission_df.to_csv(f'sexism_predictions_task3_{output_type}_{language}.csv', index=False)
        print(f"Prediction TASK3 completed. Results saved to sexism_predictions_task3_{output_type}_{language}.csv")
    return model, eval_results

## Run id

Before training, we specify the `run_id` for the predictions.

In [10]:
run_id = 2

## Training soft

In [11]:
modelname = "cardiffnlp/twitter-roberta-base-sentiment-latest"
params = {"num_train_epochs": 10,
          "learning_rate": 3e-5,
          "scheduler_type": "linear",
          "per_device_train_batch_size": 64,
          "warmup_ratio": 0.15,
          "early_stopping_patience": 5,
          "ignore_mismatched_sizes": True,
          "logging_dir": None,
          "output_dir": None,
          "language": "english",
          "output_type": "soft"
          }

_, eval_results = sexism_classification_pipeline_task3(EnTrainTask3, EnDevTask3, EnTestTask3, modelname, 6, "multi_label_classification", **params)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Icm
1,0.573900,0.572035,0.667387,0.351052,0.376693,-1.008049
2,0.461800,0.494329,0.777685,0.618686,0.673775,0.067513
3,0.433000,0.456778,0.746886,0.747636,0.744956,0.513005
4,0.394100,0.462576,0.756587,0.711031,0.728768,0.427713
5,0.359900,0.470450,0.767999,0.703308,0.729753,0.426182
6,0.329500,0.474489,0.778822,0.683186,0.722876,0.369609
7,0.305100,0.488992,0.775907,0.681600,0.720589,0.368706
8,0.278700,0.487535,0.769687,0.704979,0.732205,0.458821


2025-05-18 01:27:20,804 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:27:20,855 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
cargado 29
2025-05-18 01:28:04,054 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:28:04,104 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:28:48,013 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:28:48,064 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:29:31,697 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:29:31,746 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:30:16,126 - pyevall.evaluation 

2025-05-18 01:32:32,819 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:32:32,872 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Validation Results: {'eval_loss': 0.45677804946899414, 'eval_precision_macro': 0.7468863393794765, 'eval_recall_macro': 0.7476362369421716, 'eval_f1_macro': 0.744955964035785, 'eval_ICM': 0.5130047183586675, 'eval_runtime': 2.1632, 'eval_samples_per_second': 205.255, 'eval_steps_per_second': 3.236, 'epoch': 8.0}
2025-05-18 01:32:37,476 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:32:37,572 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Prediction TASK3 completed. Results saved to sexism_predictions_task3_soft_english.csv


In [12]:
modelname = "pysentimiento/robertuito-sentiment-analysis"
params = {"num_train_epochs": 10,
          "learning_rate": 3e-5,
          "scheduler_type": "linear",
          "per_device_train_batch_size": 64,
          "warmup_ratio": 0.15,
          "early_stopping_patience": 5, 
          "logging_dir": None,
          "output_dir": None,
          "ignore_mismatched_sizes": True,
          "language": "spanish",
          "output_type": "soft"
          }

_, eval_results = sexism_classification_pipeline_task3(SpTrainTask3, SpDevTask3, SpTestTask3, modelname, 6, "multi_label_classification", **params)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Icm
1,0.617200,0.626295,0.542201,0.478083,0.454311,-0.672965
2,0.515100,0.527315,0.691569,0.715210,0.681683,0.255776
3,0.454100,0.503983,0.757978,0.679059,0.699173,0.306111
4,0.409000,0.502652,0.749478,0.693893,0.716385,0.372053
5,0.365300,0.507726,0.752610,0.705674,0.722939,0.431276
6,0.313500,0.522273,0.757305,0.696620,0.720961,0.407712
7,0.302600,0.529355,0.756292,0.698407,0.720465,0.414936
8,0.286300,0.538432,0.752294,0.684935,0.710917,0.347686
9,0.281200,0.552494,0.764136,0.676755,0.711631,0.349265
10,0.270300,0.549366,0.753694,0.685666,0.712530,0.357182


2025-05-18 01:33:36,432 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:33:36,490 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:34:23,443 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:34:23,499 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:35:11,516 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:35:11,573 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:35:59,917 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:35:59,977 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:36:48,130 - pyevall.evaluation - INFO -   

2025-05-18 01:40:50,861 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:40:50,920 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Validation Results: {'eval_loss': 0.5077261924743652, 'eval_precision_macro': 0.7526099971768226, 'eval_recall_macro': 0.7056735830978532, 'eval_f1_macro': 0.7229389390809877, 'eval_ICM': 0.4312757991472615, 'eval_runtime': 2.5714, 'eval_samples_per_second': 190.558, 'eval_steps_per_second': 3.111, 'epoch': 10.0}
2025-05-18 01:40:56,008 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:40:56,115 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Prediction TASK3 completed. Results saved to sexism_predictions_task3_soft_spanish.csv


## Save predictions

In [18]:
if create_submision_file("ArPa Project", 1, "soft", run_id, 3, ".", "exist2025_ArPa Project"):
    print("Predictions saved")

Predictions saved


## Training hard

In [14]:
modelname = "cardiffnlp/twitter-roberta-base-sentiment-latest"
params = {"num_train_epochs": 10,
          "learning_rate": 3e-5,
          "scheduler_type": "linear",
          "per_device_train_batch_size": 64,
          "warmup_ratio": 0.15,
          "early_stopping_patience": 5,
          "ignore_mismatched_sizes": True,
          "output_dir": None,
          "logging_dir": None,
          "language": "english"
          }

_, eval_results = sexism_classification_pipeline_task3(EnTrainTask3, EnDevTask3, EnTestTask3, modelname, 6, "multi_label_classification", **params)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Icm
1,0.577800,0.582265,0.566764,0.299393,0.326982,-1.230047
2,0.457500,0.495377,0.791083,0.610118,0.674223,0.057288
3,0.430200,0.458007,0.746939,0.739445,0.739589,0.472425
4,0.392700,0.465559,0.757114,0.698768,0.722799,0.384791
5,0.354400,0.473049,0.766658,0.702546,0.729804,0.410673
6,0.332000,0.481530,0.779092,0.678549,0.721160,0.337833
7,0.302200,0.491648,0.763148,0.702075,0.729805,0.414385
8,0.273600,0.494559,0.764574,0.708221,0.732446,0.449478


2025-05-18 01:41:42,019 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:41:42,065 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:42:25,471 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:42:25,519 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:43:10,143 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:43:10,195 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:43:55,268 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:43:55,319 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:44:40,385 - pyevall.evaluation - INFO -   

2025-05-18 01:46:59,419 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:46:59,466 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Validation Results: {'eval_loss': 0.4580066502094269, 'eval_precision_macro': 0.7469387557434383, 'eval_recall_macro': 0.7394454720486007, 'eval_f1_macro': 0.7395888611919327, 'eval_ICM': 0.4724247102474496, 'eval_runtime': 2.2017, 'eval_samples_per_second': 201.658, 'eval_steps_per_second': 3.179, 'epoch': 8.0}
2025-05-18 01:47:04,155 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:47:04,245 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Prediction TASK3 completed. Results saved to sexism_predictions_task3_hard_english.csv


In [15]:
modelname = "pysentimiento/robertuito-sentiment-analysis"
params = {"num_train_epochs": 10,
          "learning_rate": 3e-5,
          "scheduler_type": "linear",
          "per_device_train_batch_size": 64,
          "warmup_ratio": 0.15,
          "early_stopping_patience": 5, 
          "output_dir": None,
          "logging_dir": None,
          "ignore_mismatched_sizes": True,
          "language": "spanish"
          }

_, eval_results = sexism_classification_pipeline_task3(SpTrainTask3, SpDevTask3, SpTestTask3, modelname, 6, "multi_label_classification", **params)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Icm
1,0.617200,0.626295,0.542201,0.478083,0.454311,-0.672965
2,0.515100,0.527315,0.691569,0.715210,0.681683,0.255776
3,0.454100,0.503983,0.757978,0.679059,0.699173,0.306111
4,0.409000,0.502652,0.749478,0.693893,0.716385,0.372053
5,0.365300,0.507726,0.752610,0.705674,0.722939,0.431276
6,0.313500,0.522273,0.757305,0.696620,0.720961,0.407712
7,0.302600,0.529355,0.756292,0.698407,0.720465,0.414936
8,0.286300,0.538432,0.752294,0.684935,0.710917,0.347686
9,0.281200,0.552494,0.764136,0.676755,0.711631,0.349265
10,0.270300,0.549366,0.753694,0.685666,0.712530,0.357182


2025-05-18 01:48:03,727 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:48:03,783 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:48:52,461 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:48:52,517 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:49:41,469 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:49:41,523 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:50:30,716 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:50:30,771 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 01:51:19,732 - pyevall.evaluation - INFO -   

2025-05-18 01:55:04,592 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:55:04,645 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Validation Results: {'eval_loss': 0.5077261924743652, 'eval_precision_macro': 0.7526099971768226, 'eval_recall_macro': 0.7056735830978532, 'eval_f1_macro': 0.7229389390809877, 'eval_ICM': 0.4312757991472615, 'eval_runtime': 2.0114, 'eval_samples_per_second': 243.616, 'eval_steps_per_second': 3.977, 'epoch': 10.0}
2025-05-18 01:55:08,878 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-18 01:55:08,981 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Prediction TASK3 completed. Results saved to sexism_predictions_task3_hard_spanish.csv


## Save predictions

In [17]:
if create_submision_file("ArPa Project", 1, "hard", run_id, 3, ".", "exist2025_ArPa Project"):
    print("Predictions saved")

Predictions saved
